In [1]:
import sqlite3
conn = sqlite3.connect(':memory:')
c = conn.cursor()

In [2]:
# Create table 1: lab_table
c.execute("CREATE TABLE lab_table (id INTEGER PRIMARY KEY AUTOINCREMENT, first TEXT, last TEXT, pay INTEGER)")
new_member = [('Sam', 'Qian', 37000),
              ('Sam', 'Galindo', 39000),
              ('Anita','Burgos', 45000),
              ('Rebecca','Vaadia',38000),
              ('Rotation','Student', 36000)]
for i in new_member:
    c.execute("INSERT INTO lab_table(first, last, pay) VALUES (?, ?, ?)", i)

In [3]:
c.execute("SELECT * FROM lab_table")
c.fetchall()

[(1, u'Sam', u'Qian', 37000),
 (2, u'Sam', u'Galindo', 39000),
 (3, u'Anita', u'Burgos', 45000),
 (4, u'Rebecca', u'Vaadia', 38000),
 (5, u'Rotation', u'Student', 36000)]

In [4]:
# Create table 2: experiments_table
c.execute("CREATE TABLE experiments_table (member_id INTEGER, experiment TEXT)")
experiments = [(1, 'td'),
              (2, 'axon tiling'),
              (2, 'dscam2'),
              (3, 'dnb'),
              (3, 'basigin',),
              (4, 'scape'),
              (4, 'compartment')]

for i in experiments:
    c.execute("INSERT INTO experiments_table(member_id, experiment) VALUES(?, ?)", i)

In [5]:
c.execute("SELECT * FROM experiments_table")
c.fetchall()

[(1, u'td'),
 (2, u'axon tiling'),
 (2, u'dscam2'),
 (3, u'dnb'),
 (3, u'basigin'),
 (4, u'scape'),
 (4, u'compartment')]

In [6]:
# cross join. the least useful and dumbest join. For each row in the first table, it populates all the rows
# of the second table
c.execute("SELECT * FROM lab_table, experiments_table")

In [7]:
# implicit inner join is more useful. For each row in the first table, it only populates the rows of the 
# second table where the value in key identifier columns are the same
c.execute("SELECT * FROM lab_table, experiments_table WHERE lab_table.id = experiments_table.member_id")
c.fetchall()

[(1, u'Sam', u'Qian', 37000, 1, u'td'),
 (2, u'Sam', u'Galindo', 39000, 2, u'axon tiling'),
 (2, u'Sam', u'Galindo', 39000, 2, u'dscam2'),
 (3, u'Anita', u'Burgos', 45000, 3, u'dnb'),
 (3, u'Anita', u'Burgos', 45000, 3, u'basigin'),
 (4, u'Rebecca', u'Vaadia', 38000, 4, u'scape'),
 (4, u'Rebecca', u'Vaadia', 38000, 4, u'compartment')]

In [8]:
# EXPLICIT inner join is the better practice for joining (gives the same results as implicit join)
c.execute("""SELECT * FROM lab_table
          JOIN experiments_table
          ON lab_table.id = experiments_table.member_id""")
c.fetchall()

[(1, u'Sam', u'Qian', 37000, 1, u'td'),
 (2, u'Sam', u'Galindo', 39000, 2, u'axon tiling'),
 (2, u'Sam', u'Galindo', 39000, 2, u'dscam2'),
 (3, u'Anita', u'Burgos', 45000, 3, u'dnb'),
 (3, u'Anita', u'Burgos', 45000, 3, u'basigin'),
 (4, u'Rebecca', u'Vaadia', 38000, 4, u'scape'),
 (4, u'Rebecca', u'Vaadia', 38000, 4, u'compartment')]

In [9]:
# join can be combined with WHERE
c.execute("""SELECT * FROM lab_table
          JOIN experiments_table
          ON lab_table.id = experiments_table.member_id
          WHERE first = 'Sam'""")
c.fetchall()

[(1, u'Sam', u'Qian', 37000, 1, u'td'),
 (2, u'Sam', u'Galindo', 39000, 2, u'axon tiling'),
 (2, u'Sam', u'Galindo', 39000, 2, u'dscam2')]

In [10]:
# while inner join only produces a row if it finds values matching in both tables,
# outer join produces a row regardless of whether it exists in the other table
c.execute("""SELECT * FROM lab_table
          OUTER LEFT JOIN experiments_table
          ON lab_table.id = experiments_table.member_id""")
c.fetchall()

[(1, u'Sam', u'Qian', 37000, 1, u'td'),
 (2, u'Sam', u'Galindo', 39000, 2, u'axon tiling'),
 (2, u'Sam', u'Galindo', 39000, 2, u'dscam2'),
 (3, u'Anita', u'Burgos', 45000, 3, u'basigin'),
 (3, u'Anita', u'Burgos', 45000, 3, u'dnb'),
 (4, u'Rebecca', u'Vaadia', 38000, 4, u'compartment'),
 (4, u'Rebecca', u'Vaadia', 38000, 4, u'scape'),
 (5, u'Rotation', u'Student', 36000, None, None)]

In [11]:
# Changing values with UPDATE
c.execute("UPDATE experiments_table SET experiment = 'campari' WHERE member_id = 1" )
c.execute("SELECT * FROM experiments_table")
c.fetchall()

[(1, u'campari'),
 (2, u'axon tiling'),
 (2, u'dscam2'),
 (3, u'dnb'),
 (3, u'basigin'),
 (4, u'scape'),
 (4, u'compartment')]

In [12]:
# Deleting rows. In practice, rows are never actually deleted (to prevent accidentally deletion), instead
# there is a column named 'deleted', that can be set to True to be excluded from querying by an app's user for example
c.execute("DELETE FROM experiments_table WHERE experiment = 'campari'" )
c.execute("SELECT * FROM experiments_table")
c.fetchall()

[(2, u'axon tiling'),
 (2, u'dscam2'),
 (3, u'dnb'),
 (3, u'basigin'),
 (4, u'scape'),
 (4, u'compartment')]

In [13]:
# Adding a new column to existing table (and filling it with a default value)
c.execute("ALTER TABLE experiments_table ADD timeline_months INTEGER DEFAULT 12")
c.execute("SELECT * FROM experiments_table")
c.fetchall()

[(2, u'axon tiling', 12),
 (2, u'dscam2', 12),
 (3, u'dnb', 12),
 (3, u'basigin', 12),
 (4, u'scape', 12),
 (4, u'compartment', 12)]

In [14]:
# Rarely use this, but deleting an entire table
# c.execute("DROP TABLE experiments_table")
c.execute("SELECT * FROM experiments_table")
c.fetchall()

[(2, u'axon tiling', 12),
 (2, u'dscam2', 12),
 (3, u'dnb', 12),
 (3, u'basigin', 12),
 (4, u'scape', 12),
 (4, u'compartment', 12)]

In [ ]:
# Joining tables to themselves with self-join. Skipped